In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("KafkaStreamToDataFrame") \
    .getOrCreate()

24/03/01 23:01:57 WARN Utils: Your hostname, Prashants-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.93 instead (on interface en0)
24/03/01 23:01:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/01 23:01:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/01 23:01:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:

# Define the schema of the JSON data
schema = StructType([
    StructField("BUSINESS NAME", StringType()),
    StructField("BUSINESS OPEN DATE", StringType()),
    StructField("BUSINESS STATUS", StringType()),
    StructField("BUSINESS CLOSE DATE", StringType()),
    StructField("NAICS CODE", IntegerType()),
    StructField("NAICS CATEGORY", StringType()),
    StructField("NAICS GROUP", StringType()),
    StructField("FULL STREET NAME", StringType()),
    StructField("CITY", StringType()),
    StructField("STATE", StringType()),
    StructField("LATITUDE", StringType()),
    StructField("LONGITUDE", StringType())
])

In [ ]:
# Read JSON data from Kafka topic
kafka_df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "big_data") \
    .load()

In [ ]:
# Parse JSON data and apply schema
parsed_df = kafka_df \
    .selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), schema).alias("parsed_value")) \
    .select("parsed_value.*")

In [16]:
parsed_df.printSchema()
parsed_df.show(2)

root
 |-- BUSINESS NAME: string (nullable = true)
 |-- BUSINESS OPEN DATE: string (nullable = true)
 |-- BUSINESS STATUS: string (nullable = true)
 |-- BUSINESS CLOSE DATE: string (nullable = true)
 |-- NAICS CODE: integer (nullable = true)
 |-- NAICS CATEGORY: string (nullable = true)
 |-- NAICS GROUP: string (nullable = true)
 |-- FULL STREET NAME: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- LATITUDE: string (nullable = true)
 |-- LONGITUDE: string (nullable = true)



+--------------------+------------------+---------------+-------------------+----------+--------------------+-----------------+----------------+-----------+-----+----------------+---------------+
|       BUSINESS NAME|BUSINESS OPEN DATE|BUSINESS STATUS|BUSINESS CLOSE DATE|NAICS CODE|      NAICS CATEGORY|      NAICS GROUP|FULL STREET NAME|       CITY|STATE|        LATITUDE|      LONGITUDE|
+--------------------+------------------+---------------+-------------------+----------+--------------------+-----------------+----------------+-----------+-----+----------------+---------------+
| HARDWARE STUDIO LLC|        12/01/2020|              O|               NULL|    444000|Building Material...| RETAIL TRADE (G)|    PERKINS RD E|BATON ROUGE|   LA|-91.022990315425| 30.34764735661|
|CONNIE'S CAKE COL...|        04/17/2023|              O|               NULL|    311000|  Food Manufacturing|MANUFACTURING (D)|      BRITISH LN|BATON ROUGE|   LA|-91.043210383604|30.349897977642|
+-------------------

In [17]:
from cassandra.cluster import Cluster

# Cassandra connections
contact_points = ['127.0.0.1']
keyspace = 'bigdata_keyspace'

# Connect to Cassandra cluster
cluster = Cluster(contact_points=contact_points)
session = cluster.connect()

# Use the keyspace
session.set_keyspace(keyspace)

In [19]:
# Prepare insert query
table_name = f'{keyspace}.businesses'
cassandra_column_names = [col.lower().replace(' ', '_') for col in parsed_df.columns]
column_names = ', '.join(cassandra_column_names)
value_placeholders = ', '.join(['%s' for _ in parsed_df.columns])

insert_query = f"INSERT INTO {table_name} ({column_names}) VALUES ({value_placeholders})"

# Deleting previous records
session.execute(
    f"TRUNCATE {table_name}"
)

for row in parsed_df.collect():
    session.execute(insert_query, tuple(row))

In [20]:
session.execute(
    f"SELECT COUNT(*) FROM {table_name}"
)